In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2                         ## To connect to Postgres Database
from dotenv import load_dotenv          ## To access .env file storing credentials
import os
import openpyxl

In [12]:
## Read the data in Python for initial screening since the data is small in size here
sales = pd.read_excel(r'D:\Self projects\E-commerce Data modelling\Data\online+retail+ii\online_retail_II.xlsx')

In [14]:
sales.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [34]:
sales.info(), sales.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


(None,
 Invoice             0
 StockCode           0
 Description      2928
 Quantity            0
 InvoiceDate         0
 Price               0
 Customer ID    107927
 Country             0
 dtype: int64)

In [ ]:
sales.duplicated().sum()

6865

#### We have 6865 duplicate rows in the dataset because this dataset contains 1 product per row meaning duplicate Transections IDs (Invoice here). So we will introduce a surrgate key to assign a primary key to the facts table

In [56]:
## Changing data types to appropriate ones
sales['Customer ID'] = sales['Customer ID'].fillna(-1).astype('Int64')

In [37]:
sales.isnull().sum()

Invoice           0
StockCode         0
Description    2928
Quantity          0
InvoiceDate       0
Price             0
Customer ID       0
Country           0
dtype: int64

## Designing STAR schema in Postgres by creating dimension and fact tables

In [3]:
#Create secure connection
load_dotenv('D:\Self projects\E-commerce Data modelling\postgres_cred.env')

try:
    post_con = psycopg2.connect(host = os.getenv('PG_HOST'), 
                            dbname = os.getenv('PG_DB'), 
                            user = os.getenv('PG_USER'), 
                            password = os.getenv('PG_PASSWORD'))
except psycopg2.Error as e:
    print(e)

In [5]:
# Get the cursor to execute SQL queries using the connection made above
cur = post_con.cursor()

# set autocommit = True to not save every query manually
post_con.set_session(autocommit=True)

In [6]:
# Create a database & connect to the new database by closing the connection to default database

try:
    cur.execute("Create database online_sales")
except psycopg2.Error as e:
    print(e)
post_con.close()

# Connect to new database just created (Database name is read in all small case letters)
try:
    post_con = psycopg2.connect(host = os.getenv('PG_HOST'), 
                            dbname = 'online_sales', 
                            user = os.getenv('PG_USER'), 
                            password = os.getenv('PG_PASSWORD'))
except psycopg2.Error as e:
    print(e)

In [7]:
# Get the cursor to execute SQL queries using the connection made above to a new database
cur = post_con.cursor()
post_con.set_session(autocommit=True)

+ **Dimensions in Sales table:** StockCode (Product ID), Customer_ID, Invoice_Date  
+ **Facts in Sales table:** Quantity, Price (These are measures)

In [ ]:
## Create Dimension tables
cur.execute("""
            CREATE TABLE dim_customer (
            customer_id INT PRIMARY KEY,
            country TEXT
            );
            """)
cur.execute("""
            CREATE TABLE dim_product (
            product_id TEXT PRIMARY KEY,
            description TEXT
            );
            """)
cur.execute("""
            CREATE TABLE dim_date (
            transection_date DATE PRIMARY KEY,
            year INT,
            month INT,
            day INT,
            weekday TEXT);
            """)

In [40]:
## Create Facts table
cur.execute("""
            CREATE TABLE fact_sales (
            sales_id SERIAL PRIMARY KEY,
            invoice_no TEXT,
            customer_id INT REFERENCES dim_customer(customer_id),
            product_id TEXT REFERENCES dim_product(product_id),
            date_id DATE REFERENCES dim_date(transection_date),
            quantity INT,
            unit_price NUMERIC,
            total_price NUMERIC);
            """)

In [44]:
sales.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [69]:
## Insert data into tables
insert_query_customers = """
    INSERT INTO dim_customer (customer_id, country) VALUES (%s, %s)
    ON CONFLICT (customer_id) DO NOTHING;
"""
insert_query_product = """
    INSERT INTO dim_product (product_id, description) VALUES (%s, %s)
    ON CONFLICT (product_id) DO NOTHING;
"""
insert_query_date = """
    INSERT INTO dim_date (transection_date, year, month, day, weekday) VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (transection_date) DO NOTHING;
"""

for row in sales.itertuples(index=False):
    cur.execute(insert_query_customers, (int(row._6), str(row.Country)))
    cur.execute(insert_query_product, (str(row.StockCode), str(row.Description)))
    cur.execute(insert_query_date, (row.InvoiceDate, row.InvoiceDate.year, row.InvoiceDate.month, row.InvoiceDate.day,row.InvoiceDate.day_name()))

In [74]:
insert_query_facts = """
    INSERT INTO fact_sales (sales_id, invoice_no, customer_id, product_id, date_id, quantity, unit_price, total_price) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (sales_id) DO NOTHING;
"""
id = 0
for row in sales.itertuples(index=False):
    id=id+1
    cur.execute(insert_query_facts, (id, str(row.Invoice), int(row._6), str(row.StockCode), row.InvoiceDate, row.Quantity, row.Price, row.Price*row.Quantity))

In [80]:
pd.read_sql("Select * from fact_sales", post_con)

C:\Users\pranj\AppData\Local\Temp\ipykernel_28336\2872162773.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("Select * from fact_sales", post_con)


,sales_id,invoice_no,customer_id,product_id,date_id,quantity,unit_price,total_price
0,1,489434,13085,85048,2009-12-01,12,6.95,83.40
1,2,489434,13085,79323P,2009-12-01,12,6.75,81.00
2,3,489434,13085,79323W,2009-12-01,12,6.75,81.00
3,4,489434,13085,22041,2009-12-01,48,2.10,100.80
4,5,489434,13085,21232,2009-12-01,24,1.25,30.00
...,...,...,...,...,...,...,...,...
525456,525457,538171,17530,22271,2010-12-09,2,2.95,5.90
525457,525458,538171,17530,22750,2010-12-09,1,3.75,3.75
525458,525459,538171,17530,22751,2010-12-09,1,3.75,3.75
525459,525460,538171,17530,20970,2010-12-09,2,3.75,7.50


In [84]:
pd.read_sql("Select * from Customer_frequency", post_con)

C:\Users\pranj\AppData\Local\Temp\ipykernel_28336\1495408117.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("Select * from Customer_frequency", post_con)


,customer_id,last_purchase_date,frequency,monetary_value
0,-1,2010-12-09,5229,1191276.06
1,12346,2010-10-04,15,-64.68
2,12347,2010-12-07,2,1323.32
3,12348,2010-09-27,1,222.16
4,12349,2010-10-28,4,2646.99
...,...,...,...,...
4379,18283,2010-11-22,6,641.77
4380,18284,2010-10-06,2,436.68
4381,18285,2010-02-17,1,427.00
4382,18286,2010-08-20,3,1188.43


In [85]:
post_con.close()